In [1]:
import poppy

In [2]:
p = 11
n = 4
f = poppy.field(p,n)
f

the field of order 14641.

In [3]:
a = poppy.array([[[0,1,2],[222,223,224]],[[2,213,905],[34,2,4573]]],f)
a,a.proj(),a.lift()

(shape 2 2 3 over the field of order 14641.,
 Array([[[   0,    1,    2],
         [ 222,  223,  224]],
 
        [[   2,  213,  905],
         [  34,    2, 4573]]], dtype=int64),
 Array([[[ 0,  0,  0,  0,  1,  0,  0,  0,  2,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  2,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  2,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  2],
         [ 2,  9,  1,  0,  3,  9,  1,  0,  4,  9,  1,  0],
         [ 0,  2,  9,  1,  0,  3,  9,  1,  0,  4,  9,  1],
         [ 9,  1,  5,  9,  9,  1,  6,  9,  9,  1,  7,  9],
         [ 4,  7,  6,  5,  4,  7,  6,  6,  4,  7,  6,  7]],
 
        [[ 2,  0,  0,  0,  4,  8,  1,  0,  3,  5,  7,  0],
         [ 0,  2,  0,  0,  0,  4,  8,  1,  0,  3,  5,  7],
         [ 0,  0,  2,  0,  9,  1,  7,  8,  8,  7,  2,  5],
         [ 0,  0,  0,  2,  6,  6,  3,  7,  1,  2,  0,  2],
         [ 1,  3,  0,  0,  2,  0,  0,  0,  8,  8,  4,  3],
         [ 0,  1,  3,  0,  0,  2,  0,  0,  5,  0, 

In [4]:
a = poppy.random((2,3,3),f,poppy.SEED)
b = poppy.random((2,3,3),f,poppy.SEED+1)

print(f'a.lift()\n{a.lift()}\n')
print(f'a.proj()\n{a.proj()}\n')
print(f'(a-a).proj()\n{(a-a).proj()}\n')
print(f'(a@b).proj()\n{(a@b).proj()}\n')
print(f'a.inv().proj()\n{a.inv().proj()}\n') 
print(f'(a@a.inv()).proj()\n{(a@a.inv()).proj()}\n')
print(f'a.trace().proj()\n{a.trace().proj()}\n')
print(f'a.det().proj()\n{a.det().proj()}\n')

a.lift()
[[[10  3  3  4  9  5  9  8  6  9  8  7]
  [ 3  3  4  3  6  6  7  9  8  2  8  8]
  [ 5  6  1  4  4  4  0  7  6  5  4  8]
  [ 3  9  7  1  8  0  3  0  6  3  7  4]
  [10 10  4  5  8  1  7  3  5 10  4  1]
  [ 1  4  3  4  5  0 10  7  9  6  2  4]
  [ 3  5  5  3  8  1 10 10  3  2  7  2]
  [ 5  6  3  5  2  7  9 10  7  5  8  7]
  [10  7  2  5  7  0  1  4  1  4  3  9]
  [ 1  4  0  2  3  0  1  1  4 10  9  3]
  [ 7  3 10  0  9  4  3  1  5  7  8  9]
  [ 0  7  3 10  9 10  7  3  4  3  1  8]]

 [[ 1  0  1  5  9  6  2  1  3 10  5  9]
  [ 1  6  4  1  9 10  9  2  4  1  4  5]
  [ 9  2  9  4  7  0  5  9  1  9  5  4]
  [ 3  2  3  9  4  5  5  5  3  5 10  5]
  [ 4  1  8  7 10  3  1  1  1  6 10  6]
  [ 8  0  0  8  9  0  6  1 10  7  2 10]
  [ 6  5  2  0  9 10  3  6  2  9  4  2]
  [ 0  6  5  2 10  4  6  3  7  4  4  4]
  [ 2  2  5  5  9  7 10  6 10  7  8  2]
  [ 1  7  6  5 10  4  3 10  7  1  2  8]
  [ 1  6  0  6  2  9  1  3  6  4  3  2]
  [10  7  2  0  5  5  7  1  7  8 10  3]]]

a.proj()
[[[ 5730 11801 10

In [5]:
f = poppy.field(2,3)
len(poppy.gl2(f)[0]),len(poppy.pgl2(f)[0]),len(poppy.sl2(f)[0]),len(poppy.psl2(f)[0])

(3528, 504, 504, 504)

In [6]:
poppy.psl2mod(7), poppy.pgl2mod(7)

((Array([  92,   93,   94,  119,  120,  121,  122,  161,  162,  163,  164,
          232,  233,  234,  274,  275,  276,  301,  302,  303,  304,  344,
          351,  358,  365,  372,  379,  386,  393,  401,  409,  442,  451,
          462,  471,  491,  504,  514,  527,  540,  555,  570,  589,  611,
          633,  638,  644,  675,  681,  743,  758,  773,  805,  813,  821,
          829,  842,  847,  864,  869,  897,  906,  926,  941,  947,  953,
          987,  997, 1010, 1023, 1088, 1093, 1115, 1135, 1148, 1158, 1171,
         1190, 1198, 1206, 1214, 1242, 1248, 1254, 1283, 1298, 1317, 1330,
         1339, 1359, 1368, 1374, 1381, 1388, 1395, 1402, 1409, 1416, 1423,
         1443, 1452, 1472, 1487, 1491, 1506, 1521, 1527, 1533, 1564, 1570,
         1578, 1611, 1619, 1632, 1655, 1668, 1673, 1690, 1695, 1718, 1725,
         1732, 1739, 1746, 1753, 1760, 1767, 1780, 1793, 1816, 1822, 1828,
         1834, 1865, 1876, 1885, 1905, 1914, 1919, 1941, 1963, 1996, 2004,
         2012, 2016, 2031

In [7]:
poppy.lps(23,17)

(Array([[51392, 82910, 21795, ..., 53789, 57393, 63292],
        [69701, 47278, 45289, ..., 75735, 41719, 25229],
        [60402, 35735, 75158, ..., 34885, 83420, 67491],
        ...,
        [33270,  4455,  8909, ..., 34120,  6664, 50627],
        [41141,  1582, 36228, ..., 17154, 73407, 30448],
        [61796,  1123, 12547, ..., 13040, 36874, 70806]], dtype=uint32),
 Array([83521, 83521, 83521, ..., 83521, 83521, 83521], dtype=uint32))